# 2 x $CO_2$

In [153]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython import display
import pymongo

from bokeh.io import output_notebook, show
from bokeh.palettes import all_palettes


from climatools.atm import *
from climatools.plot.plot import *

In [30]:
CLIENT = pymongo.MongoClient(host='localhost', port=27017)

In [108]:
MOL = 'co2'
CONC = 2 * 400e-6
COMMITNUMBER = '1013f91'
NV, DV = 1000, .001
ATMPRO = dict(mls=294, saw=257, trp=300)

## Atmosphere compositions

We first set the concentration of CO2 to that double of the usual 4e-6 ppmv.  Then we split the overall composition into those where CO2 overlaps with other gases and those where only CO2 is present.

In [13]:
atmcomp = AtmComposition.cliradlw_nongreys()
atmcomp.gasconcs['co2'] = CONC
atmcomp.gasinbands = {b:gs for b, gs in atmcomp.gasinbands.items() if MOL in gs}

In [14]:
atmcomps_olp = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b: gs}) 
                for b, gs in atmcomp.gasinbands.items() if len(gs) > 1]

In [15]:
atmcomps_bft = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b:[MOL]}) for b, _ in atmcomp.gasinbands.items()]

## CO2 only case

In [70]:
atmpro = 'mls'

In [71]:
atmbft = atmcomps_bft[0]

In [72]:
parm_cli = atmbft.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])

In [73]:
data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)

In [74]:
parms_lbl = atmbft.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], nv=NV, dv=DV)

In [75]:
parms_lbl[0].avail_commits(collection=CLIENT.lblnew.bestfit_lw)

['a22ab94']

In [76]:
data_lbl = parms_lbl[0].modeldata_pymongo(collection=CLIENT.lblnew.bestfit_lw)

In [115]:
data_cli.wgt_flux.dims, data_cli.wgt_cool.dims

(Frozen(SortedKeysDict({'i': 1, 'band': 11, 'pressure': 76})),
 Frozen(SortedKeysDict({'i': 1, 'band': 11, 'pressure': 75})))

In [116]:
data_lbl.crd_flux.dims, data_lbl.crd_cool.dims

(Frozen(SortedKeysDict({'pressure': 76})),
 Frozen(SortedKeysDict({'pressure': 75})))

In [117]:
data_lbl.wgt_flux.dims, data_lbl.wgt_cool.dims

(Frozen(SortedKeysDict({'pressure': 76, 'igg': 10})),
 Frozen(SortedKeysDict({'pressure': 75, 'igg': 10})))

## CO2 overlapped with other gases

In [84]:
atmpro = 'mls'

In [85]:
atmolp = atmcomps_olp[0]

In [88]:
parm_cli = atmolp.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])

In [89]:
data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)

In [109]:
parms_lbl = atmolp.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], dv=DV, nv=NV)

In [144]:
parms_lbl

[<class 'climatools.parameters.LBLnewOverlapParam'>
 atmpro: mls
 band: 3a
 commitnumber: None
 dv: 0.001
 molecule: {'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07}
 nv: 1000
 tsfc: 294]

In [110]:
parms_lbl[0].avail_commits(collection=CLIENT.lblnew.overlap_lw)

['a22ab94']

In [113]:
data_lbl = parms_lbl[0].modeldata_pymongo(collection=CLIENT.lblnew.overlap_lw)

In [147]:
data_cli.wgt_flux.dims, data_cli.wgt_cool.dims

(Frozen(SortedKeysDict({'i': 1, 'band': 11, 'pressure': 76})),
 Frozen(SortedKeysDict({'i': 1, 'band': 11, 'pressure': 75})))

In [148]:
data_lbl.crd_flux.dims, data_lbl.crd_cool.dims

(Frozen(SortedKeysDict({'pressure': 76})),
 Frozen(SortedKeysDict({'pressure': 75})))

In [120]:
data_lbl.wgt_flux.dims, data_lbl.wgt_cool.dims

(Frozen(SortedKeysDict({'pressure': 76, 'igg': 10})),
 Frozen(SortedKeysDict({'pressure': 75, 'igg': 10})))

### Parameter Summary

In [143]:
gs = data_lbl.param['molecule'].keys() if isinstance(data_lbl.param['molecule'], dict) else [data_lbl.param['molecule']]
ps_cliradfit = {f"({g}, {data_lbl.param['band']})": kdist_params(molecule=g, band=data_lbl.param['band']) for g in gs}
fitparams = pd.DataFrame({molgas: pd.Series(p) for molgas, p in ps_cliradfit.items()})
runparams = pd.concat([pd.Series(data_cli.param), pd.Series(data_lbl.param)], axis=1, sort=True, keys=['CLIRAD', 'LBLNEW'])
pd.concat([runparams, fitparams], axis=1, sort=True, keys=['Run parameters', 'Fit parameters']).fillna('-')

Run parameters  \
                                                                 CLIRAD   
atmpro                                                              mls   
band                                                                [3]   
commitnumber                                                    1013f91   
conc                                                                  -   
dv                                                                    -   
klin                                                                  -   
molecule               {'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07}   
ng_adju                                                               -   
ng_refs                                                               -   
nv                                                                    -   
option_compute_btable                                                 -   
option_compute_ktable                                                 -   
option_wgt_flux                                                       -   
option_wgt_k                                                          -   
ref_pts                                                               -   
tsfc                                                                294   
vmax                                                                  -   
vmin                                                                  -   
w_diffuse                                                             -   
wgt                                                                   -   

                                                                         \
                                                                 LBLNEW   
atmpro                                                              mls   
band                                                                 3a   
commitnumber                                                    a22ab94   
conc                                                                  -   
dv                                                                0.001   
klin                                                                  -   
molecule               {'co2': 0.0008, 'h2o': 'atmpro', 'n2o': 3.2e-07}   
ng_adju                                                               -   
ng_refs                                                               -   
nv                                                                 1000   
option_compute_btable                                                 -   
option_compute_ktable                                                 -   
option_wgt_flux                                                       -   
option_wgt_k                                                          -   
ref_pts                                                               -   
tsfc                                                                294   
vmax                                                                  -   
vmin                                                                  -   
w_diffuse                                                             -   
wgt                                                                   -   

                                                          Fit parameters  \
                                                               (co2, 3a)   
atmpro                                                               mls   
band                                                                  3a   
commitnumber                                                     a06b618   
conc                                                              0.0004   
dv                                                                 0.001   
klin                                                         6.37556e-24   
molecule                                                             co2   
ng_adju                                                        [0, 0, 0

### Cooling Rate Profiles

In [164]:
def pltdata_cool(dlbl=None, dcli=None):
    colors = all_palettes['Set1'][4]
    
    igg = 10
    cool_cli = dcli.wgt_cool['coolrg'].sel(i=1).sum('band')
    cool_crd = dlbl.crd_cool['coolrg']
    cool_wgt_igg = dlbl.wgt_cool['coolrg'].sel(igg=10)
    
    data = [{'label':'CLIRAD', 'srs':cool_cli,
             'line_dash':'dashed', 'line_width':5,
             'color':colors[0], 'alpha':.6},
            {'label':'CRD', 'srs':cool_crd,
             'line_dash':'solid', 'line_width':1.5,
             'marker':'circle', 'marker_size':5,
             'color':colors[2], 'alpha':1},
            {'label':f'WGT igg={igg}', 'srs':cool_wgt_igg,
             'line_dash':'solid', 'line_width':4,
             'color':colors[3], 'alpha':.3}]
    return data

In [ ]:
plt_vert_profile_bokehvert_profile_bokehvert_profile_bokehvert_profile_bokehvert_profile_bokeh

In [165]:
pltdata_cool(dlbl=data_lbl, dcli=data_cli)

[{'alpha': 0.6,
  'color': '#e41a1c',
  'label': 'CLIRAD',
  'line_dash': 'dashed',
  'line_width': 5,
  'srs': <xarray.DataArray 'coolrg' (pressure: 75)>
  array([0.631966, 0.638152, 0.659324, 0.680961, 0.700525, 0.717363, 0.729588,
         0.735686, 0.732453, 0.716947, 0.684202, 0.628396, 0.54365 , 0.424492,
         0.317688, 0.303045, 0.342646, 0.358391, 0.387386, 0.475624, 0.634482,
         0.874311, 1.105994, 1.067017, 0.847757, 0.677875, 0.555427, 0.445406,
         0.352355, 0.273927, 0.205073, 0.143538, 0.110493, 0.100442, 0.085924,
         0.062564, 0.045002, 0.034924, 0.019362, 0.003109, 0.006853, 0.032724,
         0.060553, 0.087916, 0.114007, 0.132111, 0.148505, 0.16353 , 0.177253,
         0.191506, 0.207264, 0.220938, 0.232367, 0.241548, 0.24943 , 0.256667,
         0.264225, 0.279609, 0.293608, 0.303986, 0.310101, 0.311351, 0.309374,
         0.305119, 0.29931 , 0.292636, 0.284216, 0.271401, 0.252358, 0.22712 ,
         0.200064, 0.175363, 0.1554  , 0.144344, 0.1658

In [163]:
data_cli.wgt_cool['coolrg'].sel(i=1).sum('band')

<xarray.DataArray 'coolrg' (pressure: 75)>
array([0.631966, 0.638152, 0.659324, 0.680961, 0.700525, 0.717363, 0.729588,
       0.735686, 0.732453, 0.716947, 0.684202, 0.628396, 0.54365 , 0.424492,
       0.317688, 0.303045, 0.342646, 0.358391, 0.387386, 0.475624, 0.634482,
       0.874311, 1.105994, 1.067017, 0.847757, 0.677875, 0.555427, 0.445406,
       0.352355, 0.273927, 0.205073, 0.143538, 0.110493, 0.100442, 0.085924,
       0.062564, 0.045002, 0.034924, 0.019362, 0.003109, 0.006853, 0.032724,
       0.060553, 0.087916, 0.114007, 0.132111, 0.148505, 0.16353 , 0.177253,
       0.191506, 0.207264, 0.220938, 0.232367, 0.241548, 0.24943 , 0.256667,
       0.264225, 0.279609, 0.293608, 0.303986, 0.310101, 0.311351, 0.309374,
       0.305119, 0.29931 , 0.292636, 0.284216, 0.271401, 0.252358, 0.22712 ,
       0.200064, 0.175363, 0.1554  , 0.144344, 0.165859])
Coordinates:
    i         int64 1
  * pressure  (pressure) float64 0.0003122 0.0007501 ... 977.3 1.001e+03
    layer     (pressu

## Fin